# Group: Hyacithara - Report

## System and Dependencies

All operations were tested on computers with 64 Bit multi-core processor and at least 16 GB RAM.
As operating systems, Microsoft Windows 10, Microsoft Windows 11, Manjaro Linux 23.1.3, and Fedora Linux 39 were used.
On all computers, Python 3.10 or 3.11 - e. g. Python 3.11.8 - was installed for running `pip` and the pipeline.

To make sure that all required Python packages are available, the installation gets started in the following cell.

- `!` allows for running the following command on the command line.
- `pip install -r <textfile>` (or depending on the installation `python3 -m pip …`) installs the module dependencies in the versions with which this notebook was created.
- With these module versions combined with Python 3.11.8, the notebook was tested and running. With other versions, errors can occur. One example of a problem caused by non-matching versions is the changed step naming of different `mne-bids-pipeline` versions.

In [ ]:
!pip install -r requirements.txt # install dependencies
%env PYTHONIOENCODING=utf8 # set modern encoding

## Download and Pre-process the Dataset

Now, that all required Python packages are installed, we can start to fetch all required data and perform some clean-up operations on the data.
Then, we can use the data with the MNE-BIDS pipeline.

### Preamble
To perform the pre-processing, the custom Python module gets added to the search path.
Then, the needed modules for loading the configuration file and for fetching the dataset get loaded.

To allow for plotting, a plotting library with API similar to the one of Matlab gets loaded.
This then is set to use a QT-based rendering.

In [ ]:
# Add the project source code directory to the search path
import sys
import os
sys.path.append(os.path.abspath('./src/'))

# import function to load configuration from file
from mne_bids_pipeline._config_import import _import_config as getConfig
from tools.logtools import *

# tools to get fresh data
import data_handling.data_downloader as dl
import data_handling.data_cleaner as clean

import matplotlib
matplotlib.use('qtagg')

### Load Configuration

First of all, we load the configuration for the mne-bids pipeline from the prepared file.
Note, that we have to disable checks here.
Otherwise, the import would fail if the data is not yet available like in the first run of this notebook.

In [ ]:
# set the file path of the main configuration file
bids_config_path = "./mne-bids/config/mne-bids-pipeline_data_tests.py"
# load configured settings from file
bids_cfg = getConfig(
    config_path=bids_config_path,
    check=False
)

### Dataset Download

After loading the custom data handling module, a check for the dataset existence is started.
For ensuring to have an unchanged version of the dataset, the fresh download can get enforced.

If the dataset does not exist in the location specified in the configuration file, a copy gets downloaded and extracted.
Please ensure that you have enough disk space available. About 130 GB are needed for download and extraction. Additional 40 GB should be free for running the pipeline, which can be freed by deleting the file `ds003702.zip` after successful extraction.

For existence checkup and download, there are some configuration options in the following cell.

In [ ]:
from data_handling import getDataPathFromBidsRoot

dl.CLEAN_DATA = False # if true, clears the data directory in order to force downloading a fresh copy of the data
dl.DATA_BASE_DIR = getDataPathFromBidsRoot(bids_cfg.bids_root) # get the data folder from the bids pipeline configuration
dl.VALIDATE_DATA = True # if true, checks that the downloaded zip file is the expected file

In [ ]:
if dl.CLEAN_DATA:
    clean.removeDirectory(dl.DATA_BASE_DIR)
dl.fetchData()

### Pre-processing

Once all data is downloaded and unpacked, we have to do some preprocessing, in order to be able to use the data.

For this data set, this consists mainly of two tasks:

1. Fix file links in `*.vhdr` and `*.vmrk` files. This is needed, because the files got renamed after exporting, but the original authors did not fix the file links
2. Generate a `*_events.tsv` file containing for each subject, which contains onset time, duration, and type for each labelled time frame.

In [ ]:
# import tools to patch fresh data
import data_handling.data_patcher as patch
import data_handling.convert_brainvision2bids as convert

patch.patchAllFiles(bids_cfg.bids_root)
convert.buildEventTSV(bids_cfg.bids_root)

### Validity Check

Now that we got all the data we require, we can import the config again.
This time, it is done with checks for all parameters being valid.

In [ ]:
bids_cfg = getConfig(
    config_path=bids_config_path,
)

### Electrode Coordinates
Next, we look at the used electrode coordinates.
The authors have chosen the 1010 system.
Since this is not directly given, we chose to load the coordinates of the 1005 system electrodes insdead of defining a custom 1010 system.

The unused positions get ignored.
The electrodes, for which recorded signals are given, are virtually positioned at the correct positions in the pipeline.

## Run the Pipeline

Once the preparatory steps are done, and a configuration is loaded, we can run the pipeline.

### Load Dependencies

In addition to the modules loaded as dependencies in the next cell, some dependencies were loaded in a previous cell.
This includes `mne-bids-pipeline`, which is in use for loading the configuration from the prepared file.

In [ ]:
# allow for calling mne_bids_pipeline within Python
import sys

from mne_bids import BIDSPath
from typing import Optional

### Deletion of Prior Outputs (Optional)

In case errors occur while running the pipeline, we remove the output of the previous pipeline runs.
This gets done by running the following two cells after setting `CLEAR_PIPELINE_OUTPUT` to `True`.

Note: If the value is set to `True`, the computations of the pipeline will need more time than when using some pre-processed output.

In [ ]:
CLEAR_PIPELINE_OUTPUT = False # False: Keep previous pipeline output; True: Delete all previous pipeline outputs

In [ ]:
if CLEAR_PIPELINE_OUTPUT:
    clean.removeDirectory("{}/derivatives/mne-bids-pipeline".format(bids_cfg.bids_root))


### Pipeline: Initial Pipeline Run

At this step, the preparations are finished.
Therefore, we can start running the pipeline based on the configuration file.

The initialisation should create e. g. needed directories for the subsequent steps.

In [ ]:
curr_steps = "init"
!mne_bids_pipeline --config {bids_config_path} --steps {curr_steps}

In case of Unicode encode errors when attempting to run the pipeline, make sure that the following environment variable is set:

The environment variable should already be set, if the first cell - the cell containing the module installation via `pip` - was run after starting the current Python kernel.

Remember to restart Jupyter after setting the environment variable.

### Pipeline: Pre-processing

After the initialisation, the first pre-processing steps working on the measured eeg signals get applied.

In [ ]:
curr_steps = "preprocessing/_01_data_quality"
!mne_bids_pipeline --config {bids_config_path} --steps {curr_steps}

In [ ]:
curr_steps = "preprocessing/_02_head_pos"
!mne_bids_pipeline --config {bids_config_path} --steps {curr_steps}

In [ ]:
curr_steps = "preprocessing/_03_maxfilter"
!mne_bids_pipeline --config {bids_config_path} --steps {curr_steps}

We apply a bandpass filter with a highpass frequency of 1.0 Hz and a lowpass frequency of 30.0 Hz.
The lowpass filter removes most high frequency noise from the data, including D/C noise. 
The highpass filter does some implicit detrending, and is a good practice in preparation for the ICA artefact removal.

We decided for such a narrow filter band, as our analysis is based on more narrow bandwidths.
These frequency bands in our analysis are alpha and theta oscillations.
Both bands cutoff frequencies are in the range of 3 Hz to 12 Hz. 

Thus, removing the energy of frequency bands below 1 Hz and above 30 Hz should not remove relevant information for this analysis.
Furthermore, the lowpass frequency of 30 Hz allows for downsampling the signals with sample rates of at least 60 Hz.

This is based on the Nyquist theoreme: The sample rate should be at least twice as high as the highest frequency given in the signal.
If this is not fulfilled, by aliasing the energy of the signal at frequencies higher than the Nyquist frequency gets added to the downsampled signal at other frequencies.

Since the filter curve is expected to be non-perfect in the sense of implementing a step from 0 dB amplification to -∞ dB at the cutoff frequency, a decreasing energy over the frequency above the cutoff frequency is expected.
Based on this energy decay over the frequency, we chose a higher sample rate of 100 Hz to get a slightly improved signal to noise ratio compared to a lower sample rate.

In [ ]:
curr_steps = "preprocessing/_04_frequency_filter"
!mne_bids_pipeline --config {bids_config_path} --steps {curr_steps}

We can now check the power spectrum from one of the subjects to see the effect of the filters:

In [ ]:
import mne
import matplotlib.pyplot as plt
%matplotlib widget
# checkup: print spectrum of original and filtered signals, print sample rate of new raw data file
curr_checkup_filename = f"{bids_cfg.bids_root}/derivatives/mne-bids-pipeline/sub-01/eeg/sub-01_task-SocialMemoryCuing_proc-filt_raw.fif"
curr_checkup_raw_filtered = mne.io.read_raw_fif(curr_checkup_filename, preload=True)
curr_checkup_raw_filtered.plot_psd()

Next, the epochs are being created:
We start the epochs one second before the cue object appears and end them seven seconds later, thus the time interval being -1.0 - 6.0 seconds.
Such an epoch will include a baseline of 1 second (-1s to 0s), the cue pointing down (0s to 1.5s), the cue shifting up (1.5s to 2s), the cue "looking" at the subject directly (2s to 3s), the cue shifting left or right (3s to 3.5s), the memory targets being shown (3.5s to 4s), a maintenance interval (4s to 5s), and one last second where the subject is shown a location target (5s to 6s).
(TODO: reconsider if that last second is relevant at all, maybe we could cut off after maintenance interval since we don't really know what happens during 5s and 6s: target might respond quickly, seeing feedback, or only respond after interval is over, etc.)

In [ ]:
curr_steps = "preprocessing/_05_make_epochs"
!mne_bids_pipeline --config {bids_config_path} --steps {curr_steps}

For the independent component analysis, we use the extended_infomax algorithm.

We apply a first peak-to-peak rejection to prevent high-power noise signals from influencing the ICA.
mne-bids requires us to specify a fixed threshold, for which we struggled to find a good value:
We went with 400 micro volt, but for some subjects this will reject all epochs, leading to the entire subject being exluded.
(TODO: take a closer look at those subjects, maybe there's a different issue at core?)

In [ ]:
curr_steps = "preprocessing/_06a_run_ica"
!mne_bids_pipeline --config {bids_config_path} --steps {curr_steps}

(TODO: Put this into a separate utils file?)

In [ ]:
from typing import Optional
from mne_bids import BIDSPath
# define a function which gets used in application of the ICA results to the raw data
def get_input_fnames_apply_ica(
        *,
        cfg,
        subject: str,
        session: Optional[str],
) -> dict:
    bids_basename = BIDSPath(
        subject=subject,
        session=session,
        task=cfg.task,
        acquisition=cfg.acq,
        recording=cfg.rec,
        space=cfg.space,
        datatype='eeg',
        root=cfg.deriv_root,
        check=False,
    )
    paths = dict()
    paths["ica"] = bids_basename.copy().update(suffix="ica", extension=".fif")
    paths["raw"] = bids_basename.copy().update(suffix="proc-filt_raw", extension=".fif")
    paths["components"] = bids_basename.copy().update(
        processing="ica", suffix="components", extension=".tsv"
    )
    return paths

Here, we automatically classify the components using iclabel.
We keep components labeled as "brain", as well as those labeled as "other", as this usually means that there is not enough information to clearly exlude them.
Anything labeled differently (e.g. muscle artifact, eye blink, channel noise, ...) is marked as "bad" in the .tsv file and will thus be excluded in the next step.

In [ ]:
from os.path import exists
import mne
import mne_icalabel
from mne.preprocessing import read_ica
import pandas as pd
from mne_bids_pipeline._config_utils import (
    get_subjects,
    get_sessions
)

for subject in get_subjects(bids_cfg):
    for session in get_sessions(bids_cfg):
        paths = get_input_fnames_apply_ica(cfg=bids_cfg, subject=subject, session=session)
        if not exists(paths["ica"]):
            print(formatString("ICA file not found, skipping Subject:", style=STYLE_TEXT_RED),
                  formatString(subject, style=STYLE_TEXT_BLUE))
            continue
        ica = read_ica(paths["ica"])
        raw = mne.io.read_raw_fif(paths["raw"])

        label_results = mne_icalabel.label_components(raw, ica, method="iclabel")

        print(str(ica))  # checkup print of known data about ICA
        print("\nresulting predictions:", label_results["y_pred_proba"])  # checkup print
        print("\nresulting labels:     ", label_results["labels"])  # checkup print

        labels = label_results["labels"]
        exclude_idx = [
            idx for idx, label in enumerate(labels) if label not in ["brain", "other"]
        ]
        tsv_data = pd.read_csv(paths["components"], sep="\t")

        # checkup: print old content of the file
        print("\nold tsv file content:")
        print(str(tsv_data))

        tsv_data.loc[exclude_idx, "status"] = "bad"

        # checkup: print updated content of the file
        print("\nnew tsv file content:")
        print(tsv_data)

        tsv_data.to_csv(paths["components"], sep="\t", index=False)

Here we apply the decisions from iclabel from above, rejecting any "bad" components. Before this, we might want to take a look at the components ourselves, to see if we agree with the automated classifications.

In [ ]:
curr_steps = "preprocessing/_07a_apply_ica"
!mne_bids_pipeline --config {bids_config_path} --steps {curr_steps}

In [ ]:
curr_steps = "preprocessing/_08_ptp_reject"
!mne_bids_pipeline --config {bids_config_path} --steps {curr_steps}

In [ ]:
curr_steps = "sensor"
!mne_bids_pipeline --config {bids_config_path} --steps {curr_steps}

In [ ]:
curr_steps = "source"
!mne_bids_pipeline --config {bids_config_path} --steps {curr_steps}